In [ ]:
import networkx as nx
import itertools
import numpy as np
import networkx as nx
import pprint
import pyqubo
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, Aer, transpile, assemble, execute
from qiskit.algorithms import NumPyMinimumEigensolver
import neal

def get_odd_degree_nodes(G):
    return [node for node in G.nodes() if G.degree(node) % 2 != 0]

def calculate_pairwise_distances(G, odd_nodes):
    distances = {}
    paths = {}
    for i in range(len(odd_nodes)):
        for j in range(i + 1, len(odd_nodes)):
            node_i = odd_nodes[i]
            node_j = odd_nodes[j]
            dist = nx.shortest_path_length(G, source=node_i, target=node_j, weight='weight')
            path = nx.shortest_path(G, source=node_i, target=node_j, weight='weight')
            distances[(i, j)] = dist
            paths[(i, j)] = path
    return distances, paths

def construct_W_matrix(w_ij_dict):
    size = max(max(i, j) for i, j in w_ij_dict.keys()) + 1

    # Initialize a zero matrix
    matrix = [[0]*size for _ in range(size)]

    # Fill the matrix using the dictionary
    for (i, j), value in w_ij_dict.items():
        matrix[i][j] = value
        matrix[j][i] = value  # To ensure symmetry

    return np.array(matrix)

def get_adjacency_matrix_from_graph(G, NUMBER_OF_NODES):

    adjaceny_matrix = np.zeros([NUMBER_OF_NODES, NUMBER_OF_NODES])
    for i in range(NUMBER_OF_NODES):
        for j in range(NUMBER_OF_NODES):
            temp = G.get_edge_data(i, j, default=0)
            if temp != 0:
                adjaceny_matrix[i, j] = temp["weight"]

    return adjacency_matrix


def construct_xij_matrix(NUMBER_OF_ODD_NODES):

    x_matrix = np.array(pyqubo.Array.create('x', shape=(NUMBER_OF_ODD_NODES, NUMBER_OF_ODD_NODES), vartype='BINARY'))

    for i in range(NUMBER_OF_ODD_NODES):
        x_matrix[i][i] = 0

    return pyqubo.Array(x_matrix)



def construct_hamiltonian(x_matrix, w_ij_matrix, NUMBER_OF_ODD_NODES, P_PARAMETER):

    P1 = sum((1 - sum(x_matrix[i][j] + x_matrix[j][i] for j in range(NUMBER_OF_ODD_NODES)))**2 for i in range(NUMBER_OF_ODD_NODES) for j in range(NUMBER_OF_ODD_NODES) if i != j)
    P2 = sum(x_matrix[i][k] * x_matrix[j][k] + x_matrix[k][i] * x_matrix[k][j] for i in range(NUMBER_OF_ODD_NODES) for j in range(NUMBER_OF_ODD_NODES) for k in range(NUMBER_OF_ODD_NODES) if i != k and j != k)

    H = np.sum(pyqubo.Array(w_ij_matrix*x_matrix)) + P_PARAMETER*P1+P_PARAMETER*P2

    return H

def get_optimized_xij(G):

    NUMBER_OF_NODES = len(G.nodes)

    odd_nodes = get_odd_degree_nodes(G)
    NUMBER_OF_ODD_NODES = len(odd_nodes)
    P_PARAMETER = NUMBER_OF_NODES + 2

    w_ij_dict, paths_dict =calculate_pairwise_distances(G, odd_nodes)
    w_ij_matrix = construct_W_matrix(w_ij_dict)
    x_matrix = construct_xij_matrix(NUMBER_OF_ODD_NODES)

    H = construct_hamiltonian(x_matrix, w_ij_matrix, NUMBER_OF_ODD_NODES, P_PARAMETER)

    model=H.compile()

    qubo, offset = model.to_qubo()

    bqm = model.to_bqm()
    sa = neal.SimulatedAnnealingSampler()
    sampleset = sa.sample(bqm, num_reads=10)
    decoded_samples = model.decode_sampleset(sampleset)
    best_sample = min(decoded_samples, key=lambda x: x.energy)

    return best_sample
def add_list(G):
    add_node_list = []
    best_sample = get_optimized_xij(G)
    solution_dict = best_sample.sample
    for i in solution_dict.items():
        if i[1] == 1:
            add_node_list += [(int(i[0][2]), int(i[0][5]))]
    return add_node_list

def build_graph(adjacency_matrix):
    G = nx.MultiGraph()  # Use MultiGraph to allow multiple edges between nodes

    # Add edges with weights (number of edges between nodes)
    n = len(adjacency_matrix)
    for i in range(n):
        for j in range(i, n):  # Iterate through the upper triangle of the matrix
            weight = adjacency_matrix[i][j]
            if weight > 0:
                # for _ in range(weight):
                G.add_edge(i, j, weight=weight)  # Assign weight

    return G


def is_eulerian(G):
    return nx.is_eulerian(G)

# Step 2: Check if the graph is Eulerian
def make_eulerian_naive(G,):
    add_node_list = add_list(G)  # Node pairs to add edges between

    for u, v in add_node_list:
        if G.has_edge(u, v):  # Check if there's an existing edge between u and v
            # Get the weight of the previous edge(s)
            current_weight = max(data.get("weight", 1) for data in G.get_edge_data(u, v).values())
            # Add a new edge with the new weight (e.g., increment by 1 or any other rule)
            G.add_edge(u, v, weight=current_weight)
        else:
            # If no edge exists, add it with an initial weight of 1
            G.add_edge(u, v, weight='weight')

    return G

# Step 4: Find an Eulerian circuit
def find_eulerian_circuit(G):
    circuit = list(nx.eulerian_circuit(G))
    return circuit


# Main function to solve the Chinese Postman Problem
def chinese_postman_naive(data):
    G = build_graph(data)

    if not is_eulerian(G):
        print("The graph is not Eulerian. Making it Eulerian by adding edges...")
        G = make_eulerian_naive(G)
        circuit = find_eulerian_circuit(G)
    else:
        print("The graph is already Eulerian.")

    circuit = find_eulerian_circuit(G)
    return circuit


def run(input_data,solver_params,extra_arguments):

    # Execute the algorithm
    circuit = chinese_postman_naive(input_data)

    # Display the resulting circuit
    print("Eulerian Circuit found:")
    for edge in circuit:
        print(f"Traverse from node {edge[0]} to node {edge[1]}")
    return circuit